In [1]:
import tensorflow.keras
from tensorflow.keras import layers, models, optimizers
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.compat.v1 import ConfigProto, Session
import tensorflow.keras
config = ConfigProto(allow_soft_placement=True)

config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(Session(config=config)) 

In [2]:
tensorflow.compat.v1.disable_eager_execution()

## 导入VGG16模型

In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import backend as K

model = VGG16(
    weights='imagenet',
    include_top=False
)

In [4]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

## 制作filters的激活图

In [5]:
def generate_filters_img(layer_name, filiter_index, size=150):

    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filiter_index]) #序号为filiter_index的滤波器的特征图的均值
    grads = K.gradients(loss, model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)   #l2范数
    iterate = K.function([model.input], [loss, grads])      #创建一个类似与Keras的函数API

    input_image_data = np.random.random((1, size, size, 3)) * 20 + 128
    step = 1

    for i in range(40):
        loss_value, grads_value = iterate([input_image_data])
        input_image_data += grads_value * step      #沿着梯度上升

    img = deprocess_image(input_image_data)

    return img[0]

In [6]:
def deprocess_image(x):
    x -= x.mean()               #强化表现
    x /= (x.std() + 1e-5)
    x *= 0.1

    x += 0.5                    
    x = np.clip(x, 0, 1)

    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [7]:
layer_names = [layer.name for layer in model.layers]
for layer_name in layer_names:
    try:
        size = model.get_layer(layer_name).filters       #filters的数目
    except:
        continue

    margin = 5                                           #间隔
    results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3)).astype('uint8')

    for i in range(8):
        for j in range(8):
            filters_img = generate_filters_img(layer_name, i + (j * 8), size=size)
            print('----------process on')
            horizontal_start = i * size + i * margin
            horizontal_end = horizontal_start + size
            vertical_start = j * size + j * margin
            vertical_end = vertical_start + size
            results[horizontal_start:horizontal_end, vertical_start:vertical_end, :] = filters_img

    plt.figure(figsize=(20, 20))
    plt.imshow(results)
    plt.savefig('/home/oneran/Documents/VGG16_filters_activation_map/'+layer_name)

----------process on
----------process on
----------process on
----------process on
----------process on
----------process on
----------process on
----------process on
----------process on


KeyboardInterrupt: 

In [ ]:
model.layers[1].filters